In [1]:
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException
import pandas as pd

In [2]:
client = Client("0ppfXZtnEyP5tINDcKvVSh5XLwPNHChrS7MTQ44MgS7zVXEeO1QqpICaHBr94bGa", "LyAdCy4nMkwSOGVf7lAdwjXnI7ijPoLn6WiPOMhFwn8Lc8hYvn4VtKh3755AW6aA")

In [3]:

try:
    symbol = "SOLUSDT" 
    interval = Client.KLINE_INTERVAL_1HOUR # intervalo de tiempo para las velas 
    limit = 50 # limite de velas

    klines = client.get_klines(symbol=symbol,interval=interval,limit=limit)

    barss = pd.DataFrame(klines, columns=[
        'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close time', 'Quote asset volume', 'Number of trades',
        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
    ])

    barss['Open time'] = pd.to_datetime(barss['Open time'], unit='ms') # formatea la fecha
    barss.set_index('Open time', inplace=True)

    bars = barss.iloc[[0]] # accede a la primera fila del dataframe 'penultima vela' 
    bars

except BinanceAPIException as e:
    print(f"El símbolo {symbol} no existe o no se ha podido recuperar sus datos.")
except AttributeError as e:
    print(f"El intervalo de tiempo {interval} no es valido.")
else:
    print(barss['Close'].rolling(10).mean().iloc[-1])

138.91199999999998


In [7]:
def get_latest_closed_bars(symbol: str, timeframe: str, num_bars: int) -> pd.DataFrame:

        # Define los paramertros adecuados
        interval = timeframe # intervalo de tiempo para las velas 
        limit = num_bars if num_bars > 0 else 1
        
        try:
            # Recupera los datos de las 2 ultimas velas
            klines = client.get_klines(symbol=symbol,interval=interval,limit=limit)

            if klines is None:
                print(f"El símbolo {klines} no existe o no se ha podido recuperar sus datos")
            else:
                # Crea la cabecera del DataFrame
                barss = pd.DataFrame(klines, columns=[
                    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
                    'Close time', 'Quote asset volume', 'Number of trades',
                    'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
                ])

                # Convierte la columna Open time a datatime
                barss['Open time'] = pd.to_datetime(barss['Open time'], unit='ms') 
                barss.set_index('Open time', inplace=True)

                # Renombra ciertas columnas
                barss.rename(columns={
                    'Quote asset volume':'Qte Asset Vol', 
                    'Number of trades':'Num Trades', 
                    'Taker buy base asset volume':'Taker Buy Vol', 
                    'Taker buy quote asset volume':'Taker Qte Vol'
                }, inplace=True) 

        except BinanceAPIException as e:
            print(f"El símbolo {symbol} no existe o no se ha podido recuperar sus datos.")
        except AttributeError as e:
            print(f"El intervalo de tiempo {interval} no es valido.")
        
        else:
            # si todo ok devuelve el dataframe
            return barss

In [4]:

def get_latest_tick( symbol:str) -> dict:
    try:
        tick = client.get_recent_trades(symbol=symbol,limit=1)

        if tick is None:
            print(f"No se ha podido recuperar el últinmo tick de {symbol}")

    except BinanceAPIException as e:
        print(f"No se ha podido recuperar el ultimo tick, el símbolo {symbol} no es correcto.")
    except BinanceRequestException as e:
        print(f"Algo no ha salido bien a la hora de recuperar el último tick de {symbol}")
    except Exception as e:
        print(f"Algo no ha salido bien a la hora de recuperar el último tick, error: {e}")
        
    else:
        return tick

In [6]:
get_latest_tick("SOLUSDT")

[{'id': 729978535,
  'price': '140.83000000',
  'qty': '0.49700000',
  'quoteQty': '69.99251000',
  'time': 1728485180947,
  'isBuyerMaker': False,
  'isBestMatch': True}]

In [9]:
# Obtiene las últimas velas de 1 hora de SOL/USDT
bars_prueba = get_latest_closed_bars(symbol="SOLUSDT", timeframe=client.KLINE_INTERVAL_1HOUR, num_bars=50)
bars_prueba['Close'] = pd.to_numeric(bars_prueba['Close'])
bars_prueba['Close'][-50:].mean()


141.407

In [11]:

bars_prueba['Close'].rolling(10).mean()

Open time
2024-10-08 15:00:00        NaN
2024-10-08 16:00:00        NaN
2024-10-08 17:00:00        NaN
2024-10-08 18:00:00        NaN
2024-10-08 19:00:00        NaN
2024-10-08 20:00:00        NaN
2024-10-08 21:00:00        NaN
2024-10-08 22:00:00        NaN
2024-10-08 23:00:00        NaN
2024-10-09 00:00:00    143.628
2024-10-09 01:00:00    143.831
2024-10-09 02:00:00    143.842
2024-10-09 03:00:00    143.902
2024-10-09 04:00:00    143.942
2024-10-09 05:00:00    143.988
2024-10-09 06:00:00    143.896
2024-10-09 07:00:00    143.791
2024-10-09 08:00:00    143.759
2024-10-09 09:00:00    143.735
2024-10-09 10:00:00    143.575
2024-10-09 11:00:00    143.412
2024-10-09 12:00:00    143.206
2024-10-09 13:00:00    142.923
2024-10-09 14:00:00    142.722
2024-10-09 15:00:00    142.585
2024-10-09 16:00:00    142.315
2024-10-09 17:00:00    142.115
2024-10-09 18:00:00    141.879
2024-10-09 19:00:00    141.666
2024-10-09 20:00:00    141.280
2024-10-09 21:00:00    140.946
2024-10-09 22:00:00    140.68

In [10]:

bars_prueba['Close'].rolling(10).mean().iloc[-1]

138.882

In [20]:
exchange_info = client.get_symbol_info("ADABTC")
exchange_info_filters = exchange_info['filters']

for fliter in exchange_info_filters:
    if fliter['filterType'] == 'LOT_SIZE':
        minQty = fliter['minQty']

print(minQty)


0.10000000
